# HW2 Problem 4: Multi-Commodity Flow

We define the following sets:
- Directed graph $G = (V, E)$ with $V = \{s_1, s_2, s_3, x, y, t_1, t_2, t_3\}$

- Commodities $K=\{1,2,3\}$, where commodity $k$ must be shipped from $s_k$ to $t_k$ with demand $d_k$:
$$(s_1, t_1, d_1)=(s_1, t_1, 9),\,\,(s_2, t_2, d_2)=(s_2, t_2, 2),\,\,(s_3, t_3, d_3)=(s_3, t_3, 3)$$

- Set of edges $E$ with capacity $u_e$ and unit cost $c_e$:
$$E = \{(s_1, t_1), (s_1,x), (s_2, x), (s_3, x), (s_3, t_3), (x,y), (y, t_1), (y, t_2), (y, t_3)\}$$

<!-- | Edge $e$      | Capacity $u_e$   | Cost $c_e$   |
|---------------|------------------|--------------|
| $(s_1, t_1)$  | 7                | 3            |
| $(s_1, x)$    | 5                | 1            |
| $(s_2, x)$    | 3                | 2            |
| $(s_3, x)$    | 5                | 2            |
| $(x, y)$      | 14               | 2            |
| $(y, t_1)$    | 3                | 1            |
| $(y, t_2)$    | 3                | 2            |
| $(y, t_3)$    | 4                | 2            |
| $(s_3, t_3)$  | 5                | 5            | -->


---
### Optimization Problem Formulation

Objective:
$$
\min \sum_{e\in E}\sum_k c_e x_{e,k}
$$

Subject to:

- Capacity constraints $$\sum_k x_{e,k} \le u_e \qquad \forall e\in E$$

- Flow conservation $$\sum_{(u,v)\in E} x_{u,v,k} - \sum_{(v,w)\in E}w_{v,w,k} = \begin{cases}
  d_k, &\qquad \text{if } v = s_k,\\
  -d_k, &\qquad \text{if } v = t_k,\\
  0, &\qquad \text{otherwise.}
\end{cases} \qquad \forall v \in V, \,\forall k\in K$$

- Nonnegativity $$x_{e,k} \ge 0$$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path

from pyomo.environ import *
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
# Set of commodities
k = set(range(1,4))

# Set of nodes
nodes = {'s1', 's2', 's3', 'x', 'y', 't1', 't2', 't3'}

# Set of demands and sinks for each commodity
d = {
    ('s1', 1):9,
    ('s2', 2):2,
    ('s3', 3):3,
    ('t1', 1):-9,
    ('t2', 2):-2,
    ('t3', 3):-3
}

# Directions for each commodity
directions = {
    ('s1','t1'):1,
    ('s1','x'):1,
    ('s2','x'):1,
    ('s3','x'):1,
    ('s3','t3'):1,
    ('x','y'):1,
    ('y','t1'):1,
    ('y','t2'):1,
    ('y','t3'):1,
}

# Set of edges and capacities
u = {
    ('s1', 't1'):7,
    ('s1', 'x'):5,
    ('s2', 'x'):3,
    ('s3', 'x'):5,
    ('s3', 't3'):5,
    ('x', 'y'):14,
    ('y', 't1'):3,
    ('y', 't2'):3,
    ('y', 't3'):4
}

# Set of edges and costs
c = {
    ('s1', 't1'):3,
    ('s1', 'x'):1,
    ('s2', 'x'):2,
    ('s3', 'x'):2,
    ('s3', 't3'):5,
    ('x', 'y'):2,
    ('y', 't1'):1,
    ('y', 't2'):2,
    ('y', 't3'):2
}

In [3]:
# Create a Pyomo model
m = pe.ConcreteModel()

# Define sets and parameters
m.k = pe.Set(initialize=k)
m.nodes = pe.Set(initialize=nodes)

m.d = pe.Param(m.nodes, m.k, initialize=d, default=0)
m.directions = pe.Param(m.nodes, m.nodes, initialize=directions, default = 0)
m.u = pe.Param(m.nodes, m.nodes, initialize=u, default = 0)
m.c = pe.Param(m.nodes, m.nodes, initialize=c, default = 0)

# Decision variables
m.x = pe.Var(m.nodes, m.nodes, m.k, domain=pe.NonNegativeReals)

# Objective function: Minimize total cost
obj = sum(sum(m.c[i,j] * m.x[i,j,k] for k in m.k) for i in m.nodes for j in m.nodes)
m.OBJ = pe.Objective(expr=obj, sense=pe.minimize)

# Capacity Constraint
def capacity_rule(m, i, j):
    return sum(m.x[i,j,k] for k in m.k) <= m.u[i,j]

m.CapacityConstraint = pe.Constraint(m.nodes, m.nodes, rule=capacity_rule)

# Flow conservation
def flow_rule(m,i,k):
    inflow = sum(m.x[i,p,k] for p in m.nodes)
    outflow = sum(m.x[w,i,k] for w in m.nodes)
    return inflow - outflow == m.d[i,k]

m.flow_constraint =  pe.Constraint(m.nodes,m.k,rule=flow_rule)


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [4]:
solver = po.SolverFactory('gurobi', solver_io='python')
results = solver.solve(m, tee=True)

Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 88 rows, 192 columns and 528 nonzeros (Min)
Model fingerprint: 0xb7fed892
Model has 27 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]

Presolve removed 88 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.600000000e+01


In [6]:
print("Optimal value: ", pe.value(m.OBJ))
for i in m.k:
    print(f"Commodity {i}:")
    for j in m.nodes:
        for l in m.nodes:
            if pe.value(m.x[j,l,i]) > 0:
                print(f"  Flow from {j} to {l}: {pe.value(m.x[j,l,i])}")

Optimal value:  56.0
Commodity 1:
  Flow from x to y: 2.0
  Flow from s1 to t1: 7.0
  Flow from s1 to x: 2.0
  Flow from y to t1: 2.0
Commodity 2:
  Flow from x to y: 2.0
  Flow from s2 to x: 2.0
  Flow from y to t2: 2.0
Commodity 3:
  Flow from s3 to t3: 3.0
